In [ ]:
# start sedona

from sedona.spark import SedonaContext

config = SedonaContext.builder(). \
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all/'). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.5_2.12:1.7.1,'
           'org.datasyslab:geotools-wrapper:1.7.1-28.5,'
           'edu.ucar:cdm-core:5.4.2'). \
    config('spark.driver.memory', '10g'). \
    config('spark.driver.maxResultSize', '5g'). \
    config('spark.network.timeout', '1000s'). \
    getOrCreate()

sedona = SedonaContext.create(config)

25/05/13 15:26:07 WARN Utils: Your hostname, cs-spatial-501 resolves to a loopback address: 127.0.0.1; using 128.101.33.153 instead (on interface eno1)
25/05/13 15:26:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
https://artifacts.unidata.ucar.edu/repository/unidata-all/ added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/huan1531/.ivy2/cache
The jars for the packages stored in: /home/huan1531/.ivy2/jars
org.apache.sedona#sedona-spark-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
edu.ucar#cdm-core added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-615007b0-a7ea-435b-a39d-d19dee4cb882;1.0
	confs: [default]


:: loading settings :: url = jar:file:/export/scratch/huan1531/sedona-exp/venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-3.5_2.12;1.7.1 in central
	found org.apache.sedona#sedona-common;1.7.1 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.14 in central
	found com.uber#h3;4.1.1 in central
	found net.sf.geographiclib#GeographicLib-Java;1.52 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.2 in central
	found org.checkerframework#checker-qual;3.10.0 in central
	found com.google.error

25/05/13 15:26:22 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
df = sedona.read.format("binaryFile").load("data/2m_temperature-2020-01-0*.nc")

In [3]:
df.show()

+--------------------+--------------------+-------+--------------------+
|                path|    modificationTime| length|             content|
+--------------------+--------------------+-------+--------------------+
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|
+--------------------+--------------------+-------+--------------------+



In [10]:
from pyspark.sql import functions as f

df = df.withColumn("raster", f.expr("RS_FromNetCDF(content, 't2m', 'longitude', 'latitude')"))

In [5]:
df.show()

+--------------------+--------------------+-------+--------------------+--------------------+
|                path|    modificationTime| length|             content|              raster|
+--------------------+--------------------+-------+--------------------+--------------------+
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44 46 0D 0...|GridCoverage2D["g...|
|file:/export/scra...|2025-05-09 12:21:...|2097704|[89 48 44

In [11]:
df.createOrReplaceTempView("NetCDF")
sedona.sql("SELECT RS_NetCDFInfo(content) as band FROM NetCDF").show(truncate=False)

+---------------------------------+
|band                             |
+---------------------------------+
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
|t2m(latitude=721, longitude=1440)|
+---------------------------------+



In [12]:
sedona.sql("SELECT RS_MetaData(raster) FROM NetCDF").show(truncate=False)

+---------------------------------------------------------------------+
|rs_metadata(raster)                                                  |
+---------------------------------------------------------------------+
|{-180.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-180.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-180.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-180.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-180.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-180.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-180.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-180.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
|{-180.125, 90.125, 1440, 721, 0.25, -0.25, 0.0, 0.0, 0, 1, 1440, 721}|
+---------------------------------------------------------------------+



In [ ]:
# All in one

from pyspark.sql import functions as f

df = (
    sedona.read.format("binaryFile")
    .load("data/2m_temperature-2020-01-0*.nc")
    .withColumn("raster", f.expr("RS_FromNetCDF(content, 't2m', 'longitude', 'latitude')"))
    # .selectExpr("RS_FromNetCDF(content, 't2m', 'longitude', 'latitude') as raster")
)
df.show()

In [ ]:
sedona.stop()